In [4]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import time

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [41]:
train_data_dir = 'D:/chest_xray/train'
test_data_dir =  'D:/chest_xray/test'
validation_data_dir = 'D:/chest_xray/val'

In [9]:
img_width, img_height = 150, 150

In [10]:
nb_train_samples = 2000 

In [11]:
nb_validation_samples = 800 

In [12]:
batch_size = 16

In [13]:
epochs = 5

In [14]:
test_generator_samples = 300

In [15]:
test_batch_size = 25 

In [16]:
input_shape = (img_width, img_height, 3)

In [52]:
model = Sequential()

In [53]:
model.add( Input(shape = (img_width,img_height, 3) ))

In [54]:

model.add(Conv2D(filters=64,kernel_size=3,strides= 2,padding='same',activation= 'relu' ))

In [55]:
model.add(MaxPool2D())

In [56]:

model.add(Conv2D(filters=32,kernel_size=3,activation= 'relu' ))

In [57]:
model.add(MaxPool2D())

In [58]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 35, 35, 32)        18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
Total params: 20,256
Trainable params: 20,256
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.add(Flatten())

In [60]:
model.add(Dense(256, activation='relu'))

In [61]:
model.add(Dropout(rate=0.2))

In [62]:
model.add(Dense(64, activation='relu'))

In [63]:
model.add(Dropout(rate=0.5))

In [64]:

model.add(Dense(1, activation='sigmoid'))

In [65]:
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 75, 75, 64)        1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 35, 35, 32)        18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 9248)              0         
                                                                 
 dense_3 (Dense)             (None, 256)              

In [67]:
tr_dtgen = ImageDataGenerator(
                              rescale=1. / 255,      
                              shear_range=0.2,       
                              zoom_range=0.2,
                              horizontal_flip=True,
                              )

In [68]:
train_generator = tr_dtgen.flow_from_directory(
                                               train_data_dir,       
                                               target_size=(img_width, img_height),  
                                               batch_size=batch_size,  
                                               class_mode='binary')

Found 5216 images belonging to 2 classes.


In [69]:
val_dtgen = ImageDataGenerator(rescale=1. / 255)

In [70]:
validation_generator = val_dtgen.flow_from_directory(
                                                     validation_data_dir,
                                                     target_size=(img_width, img_height),   
                                                     batch_size=batch_size,    
                                                     class_mode='binary'  
                                                     )

Found 16 images belonging to 2 classes.


In [74]:
start = time.time()
history = model.fit_generator(
                              generator = train_generator,
                              steps_per_epoch=nb_train_samples // batch_size,
                              epochs=2,
                              validation_data=validation_generator,
                              verbose = 1,
                              validation_steps=nb_validation_samples // batch_size
                              )

end = time.time()
(end - start)/60

Epoch 1/2


C:\Users\163100\AppData\Local\Temp\ipykernel_7136\665502559.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


125/125 [==============================] - 26s 204ms/step - loss: 0.2138 - accuracy: 0.9225 - val_loss: 0.5118 - val_accuracy: 0.7500
Epoch 2/2
125/125 [==============================] - 25s 203ms/step - loss: 0.2267 - accuracy: 0.9090


0.8549242138862609

In [77]:
result = model.evaluate(
                        validation_generator,
                        verbose = 1,
                        steps = 4       
                        )
result 

4/4 [==============================] - 0s 4ms/step - loss: 1.0543 - accuracy: 0.6250


[1.0542699098587036, 0.625]

In [78]:
pred = model.predict(validation_generator, steps = 2)
pred[:5]

2/2 [==============================] - 0s 11ms/step


array([[0.9961701],
       [0.983559 ],
       [0.9962295],
       [0.834793 ],
       [0.9998859]], dtype=float32)